In [2]:
import numpy as np
import pandas as pd

In [3]:
!wget http://wikipedia2vec.s3.amazonaws.com/models/zh/2018-04-20/zhwiki_20180420_300d.txt.bz2

--2020-08-26 15:57:08--  http://wikipedia2vec.s3.amazonaws.com/models/zh/2018-04-20/zhwiki_20180420_300d.txt.bz2
Resolving wikipedia2vec.s3.amazonaws.com (wikipedia2vec.s3.amazonaws.com)... 52.219.16.45
Connecting to wikipedia2vec.s3.amazonaws.com (wikipedia2vec.s3.amazonaws.com)|52.219.16.45|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 868159128 (828M) [application/x-bzip2]
Saving to: ‘zhwiki_20180420_300d.txt.bz2’

zhwiki_20180420_300 100%[===================>] 827.94M  1.89MB/s    in 5m 35s  

2020-08-26 16:02:45 (2.47 MB/s) - ‘zhwiki_20180420_300d.txt.bz2’ saved [868159128/868159128]



In [4]:
!bzip2 -d zhwiki_20180420_300d.txt.bz2

In [5]:
from wikipedia2vec import Wikipedia2Vec
model_ext = Wikipedia2Vec.load_text('zhwiki_20180420_300d.txt')

In [7]:
import jieba

In [8]:
import shutil
import os
wiki_dict_path = '../data/wiki-dict.txt'
if os.path.exists(wiki_dict_path):
    os.remove(wiki_dict_path)

In [9]:
with open(wiki_dict_path, 'w') as f:
    for idx, w in enumerate(model_ext.dictionary):
        if hasattr(w, "text"):
            #### skip entity
            f.write(u"{}\n".format(w.text))
        if idx > 0 and idx % 100000 == 0:
            print("write num {}".format(idx))
            #break


write num 100000
write num 200000
write num 300000
write num 400000
write num 500000
write num 600000
write num 700000
write num 800000
write num 900000
write num 1000000
write num 1100000
write num 1200000
write num 1300000
write num 1400000
write num 1500000


In [10]:
jieba.load_userdict(wiki_dict_path)

Building prefix dict from the default dictionary ...
Loading model from cache /tmp/jieba.cache
Loading model cost 1.264 seconds.
Prefix dict has been built successfully.


In [11]:
data = pd.read_csv("../data/DMSC.csv", index_col = 0)

/home/svjack/anaconda3/envs/py36/lib/python3.6/site-packages/numpy/lib/arraysetops.py:580: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


In [12]:
from tqdm import tqdm_notebook
import re
with open("nlp-master/stopwords.txt", "r") as f:
    stopwords = list(map(lambda line: line.strip("\n") ,f.readlines()))
def cut(data, labels, stopwords):
    result = []
    new_labels = []
    for index in tqdm_notebook(range(len(data))):
        comment = clean_str(data[index])
        label = labels[index]
        seg_list = jieba.cut(comment, cut_all = False, HMM = True)
        seg_list = list(filter(lambda x: x.strip("\n") if (x not in stopwords and len(x) > 1) else None, seg_list))
        if len(seg_list) > 1:
            result.append(seg_list)
            new_labels.append(label)
    return result, new_labels
def clean_str(line):
    line.strip('\n')
    line = re.sub(r"[^\u4e00-\u9fff]", "", line)
    line = re.sub(
        "[0-9a-zA-Z\-\s+\.\!\/_,$%^*\(\)\+(+\"\')]+|[+——！，。？、~@#￥%……&*（）<>\[\]:：★◆【】《》;；=?？]+", "", line)
    return line.strip()

In [13]:
def cut_iter(data, stopwords):
    assert type(data) == type(pd.Series([]))
    for index in tqdm_notebook(range(len(data))):
        comment = clean_str(data.iloc[index])
        seg_list = jieba.cut(comment, cut_all = False, HMM = True)
        seg_list = list(filter(lambda x: x.strip("\n") if (x not in stopwords and len(x) > 1) else None, seg_list))
        if len(seg_list) > 1:
            yield seg_list

In [14]:
comment_cut_iter = cut_iter(data['Comment'], stopwords)

In [15]:
comment_cut_path = "../data/douban_comment_cut.txt"
with open(comment_cut_path, 'w') as f:
    for idx, seg_list in enumerate(comment_cut_iter):
        f.write(u"{}\n".format(" ".join(seg_list)))

/home/svjack/anaconda3/envs/py36/lib/python3.6/site-packages/ipykernel_launcher.py:2: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  
/home/svjack/anaconda3/envs/py36/lib/python3.6/site-packages/ipykernel_launcher.py:3: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  This is separate from the ipykernel package so we can avoid doing imports until


In [16]:
split_output_dir = "../data/douban_comment_cut_split"
import shutil
if os.path.exists(split_output_dir):
    shutil.rmtree(split_output_dir)
os.mkdir(split_output_dir)

In [17]:
from fsplit.filesplit import FileSplit
fs = FileSplit(file = comment_cut_path, output_dir = split_output_dir, splitsize=10 * (2**20))
fs.split()

In [18]:
def single_comment_words_construct(comment_cut_path):
    comment_distinct_words = set([])
    with open(comment_cut_path, "r") as f:
        idx = 0
        while True:
            line = f.readline()
            if line:
                idx += 1
                if idx > 0 and idx % 100000 == 0:
                    print("read num {} set size {}".format(idx, len(comment_distinct_words)))
                for w in line.split(" "):
                    comment_distinct_words.add(w)
            else:
                break
    return comment_distinct_words

In [19]:
from functools import reduce
from glob import glob
import os
comment_distinct_words = reduce(lambda a, b: a.union(b) ,map(single_comment_words_construct, glob(os.path.join(split_output_dir, "*"))))

read num 100000 set size 61204
read num 100000 set size 71870
read num 100000 set size 70314
read num 100000 set size 64985
read num 100000 set size 71134
read num 100000 set size 64918
read num 100000 set size 67528
read num 100000 set size 66566
read num 100000 set size 59286
read num 100000 set size 63806
read num 100000 set size 59520
read num 100000 set size 69606
read num 100000 set size 59126


In [20]:
from gensim.models import KeyedVectors
KeyedVectors_ext = KeyedVectors(vector_size=model_ext.get_word_vector(u"中国").shape[0])
flush_size = 10000 
words, vectors = [], []
def get_wrapper(word_text):
    req = None
    try:
        req = model_ext.get_word_vector(word_text)
    except:
        pass
    return req
for word_text in comment_distinct_words:
    vec = get_wrapper(word_text)
    if vec is not None:
        words.append(word_text)
        vectors.append(vec)
    if len(words) >= flush_size:
        print("flush {} into".format(len(words)))
        KeyedVectors_ext.add(words, vectors)
        words, vectors = [], []
KeyedVectors_ext.add(words, vectors)
len(KeyedVectors_ext.vocab), len(comment_distinct_words)

flush 10000 into
flush 10000 into
flush 10000 into
flush 10000 into
flush 10000 into
flush 10000 into
flush 10000 into
flush 10000 into
flush 10000 into
flush 10000 into
flush 10000 into
flush 10000 into
flush 10000 into


(130156, 328151)

In [21]:
from gensim.models import Word2Vec
w2v_model = Word2Vec(size = KeyedVectors_ext.wv[u"中国"].shape[0], min_count=1)
w2v_model.build_vocab([list(comment_distinct_words)], update=False)
len(w2v_model.wv.vocab)

/home/svjack/anaconda3/envs/py36/lib/python3.6/site-packages/ipykernel_launcher.py:2: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  


328151

In [22]:
for k in KeyedVectors_ext.vocab.keys():
    w2v_model.wv.syn0[w2v_model.wv.vocab[k].index] = KeyedVectors_ext.wv[k]

/home/svjack/anaconda3/envs/py36/lib/python3.6/site-packages/ipykernel_launcher.py:2: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  
/home/svjack/anaconda3/envs/py36/lib/python3.6/site-packages/ipykernel_launcher.py:2: DeprecationWarning: Call to deprecated `syn0` (Attribute will be removed in 4.0.0, use self.vectors instead).
  


In [25]:
def iter_one_time(w2v_model, file_path = None):
    if file_path is None:
        file_path = np.random.choice(glob(os.path.join(split_output_dir, "*")))
    sentences = read_tiny_file_to_sentences(file_path)
    w2v_model.train(sentences, epochs=5, total_examples=len(sentences))
    return w2v_model

def read_tiny_file_to_sentences(tiny_file):
    with open(tiny_file, "r") as f:
        return list(map(lambda x: x.strip().split(" ") ,f.readlines()))

In [ ]:
run_times = int(1e6)
save_every = 3
w2v_save_path = "../data/w2v_douban.model"
if os.path.exists(w2v_save_path):
    os.remove(w2v_save_path)
for run_time in range(run_times):
    w2v_model = iter_one_time(w2v_model)
    if run_time > 0 and run_time % save_every == 0:
        w2v_model.save(w2v_save_path)
        print("save {} to {}".format(run_time ,w2v_save_path))

save 3 to ../data/w2v_douban.model
save 6 to ../data/w2v_douban.model
save 9 to ../data/w2v_douban.model
save 12 to ../data/w2v_douban.model
save 15 to ../data/w2v_douban.model
save 18 to ../data/w2v_douban.model
save 21 to ../data/w2v_douban.model
save 24 to ../data/w2v_douban.model
save 27 to ../data/w2v_douban.model
save 30 to ../data/w2v_douban.model
save 33 to ../data/w2v_douban.model
save 36 to ../data/w2v_douban.model
save 39 to ../data/w2v_douban.model
save 42 to ../data/w2v_douban.model
save 45 to ../data/w2v_douban.model
save 48 to ../data/w2v_douban.model
